In [1]:
import numpy as np

# Домашнее задание 

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

### Novelty

Например, для рекомендаций фильмов:  

Берем топ N рекомендованых для юзера фильмов, и считаем их средний рейтинг(значения из общего рейтинга)

### MAE и RMSE  

Предсказываем рейтинги фильмов с минимальной ошибкой MAE или RMSE, сортируем по убыванию, берем топ N

### User coverage

Выбираем порог рейтинга фильма. Если для юзера был предложен фильм выше порога, то флаг в единицу. Итого суммируем флаги и делим на количество оцененных пользователей

___

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [1]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1

In [2]:
recommended_list = [143,156, 1134, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 27, 991]

In [3]:
[hit_rate_at_k(recommended_list, bought_list, k) for k in range(len(recommended_list))]

[0, 0, 0, 0, 1, 1, 1, 1, 1]

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

In [5]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [8]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list[:k])
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def money_precision_at_k_(recommended_list, bought_list, prices_recommended, k=5):
    
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [7]:
precision(recommended_list, bought_list)

0.2

In [8]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [12]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    recall = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    return recall

In [10]:
recall(recommended_list, bought_list)

0.5

In [ ]:
([143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], [521, 32, 143, 991])

In [13]:
recall_at_k(recommended_list,bought_list,k=10)

0.5

---

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка рекомендаций

In [5]:
recommended_list = [4,22,21,1,234,232432,234234,666] #id товаров
bought_list = [1,221,3,4,5,6,7,8,9]

In [10]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    amount_relevant = len(relevant_indexes)
    if amount_relevant == 0:
        return 0
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant + 1) 
                for index_relevant in relevant_indexes])
    return sum_/amount_relevant

In [9]:
ap_k(recommended_list, bought_list, k=5)

(array([0, 3]),)
[0 3]


0.25

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [37]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[156, 32, 143],  # юзер 1
                       [143, 156, 14, 43, 11], # юзер 2
                       [533,2]] # юзер 3

In [26]:
def map_k(recommended_list, bought_list, k=5):
    
    if len(recommended_list) != len(bought_list):
        raise Exception("lists length not equal")
    
    result = np.sum([ap_k(rec, bought, k) for rec, bought in 
                     zip(recommended_list, bought_list)]) / len(recommended_list)
    
    return result

In [27]:
map_k(recommended_list_3_users, bought_list_3_users)

0.27777777777777773

### Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i+1)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [1]:
recommended_list = [4,22,21,1,234,232432,234234,666] #id товаров
bought_list = [1,221,3,4,5,6,7,8,9]

In [35]:
# по желанию
def ndcg_at_k(rec_list, bought_list, k=5):
    bought_list = np.array(bought_list[:k])
    recommended_list = np.array(rec_list[:k])
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    ideal_den = np.array([i if i < 3 else np.log2(i) for i in range(1, k + 1)])
    ideal_ones = np.ones_like(ideal_den)
    
    n_ideal_ones = np.zeros_like(ideal_den)
    n_ideal_ones[relevant_indexes] = 1
    dcg = np.sum(n_ideal_ones / ideal_den) / k
    
    ideal_dcg = np.sum(ideal_ones / ideal_den) / k
    return dcg / ideal_dcg
        

In [36]:
ndcg_at_k(recommended_list, bought_list)

0.489938890671454

### Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [75]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    result = []
    for rec, bought in zip(recommended_list, bought_list):
        relevant = np.nonzero(np.isin(rec[:k], bought[:k]))[0]
        k_u = 1 / (relevant[0] + 1) if relevant.any() else 0
        result.append(k_u)
    return np.mean(result)

In [76]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.5277777777777778